In [ ]:
import tkinter as tk
from tkinter import messagebox
import random

parties_gagnees = 0
parties_perdues = 0

def choisir_mot():
    fichier = open("mots.txt", encoding="utf-8")
    mots = []
    for ligne in fichier:
        ligne = ligne.strip()
        if ligne:
            mots.append(ligne.lower())
    fichier.close()
    return random.choice(mots)

def afficher_mot_cache(mot, lettres_trouvees):
    return " ".join([lettre if lettre in lettres_trouvees else "_" for lettre in mot])

def choisir_lettre(valeur):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    return valeur in alphabet

def dessiner_support():
    canvas.delete("all")
    canvas.create_line(100, 180, 200, 180, fill="dim gray", width=4)          
    canvas.create_line(150, 180, 150, 20, fill="saddle brown", width=4)     
    canvas.create_line(150, 20, 200, 20, fill="peru", width=4)             
    canvas.create_line(200, 20, 200, 40, fill="black", width=2)              

def dessiner_pendu(erreurs):
    if erreurs == 1:
        canvas.create_oval(185, 40, 215, 70, outline="black", width=2)               
    elif erreurs == 2:
        canvas.create_line(200, 70, 200, 110, fill="black", width=2)             
    elif erreurs == 3:
        canvas.create_line(200, 80, 180, 100, fill="black", width=2)              
    elif erreurs == 4:
        canvas.create_line(200, 80, 220, 100, fill="black", width=2)             
    elif erreurs == 5:
        canvas.create_line(200, 110, 180, 140, fill="black", width=2)                
    elif erreurs == 6:
        canvas.create_line(200, 110, 220, 140, fill="black", width=2)               

def verifier_lettre():
    global lettres_trouvees, tentatives_restantes

    lettre = entree_lettre.get().lower()
    entree_lettre.delete(0, tk.END)

    if len(lettre) != 1 or not choisir_lettre(lettre):
        message.set("Veuillez n'entrer qu'une seule lettre.")
        return

    if lettre in lettres_trouvees:
        message.set(f"Vous avez déjà essayé la lettre '{lettre}'.")
        return

    lettres_trouvees.add(lettre)

    if lettre not in mot_a_trouver:
        tentatives_restantes -= 1
        message.set(f"FAUX ! Il vous reste {tentatives_restantes} tentatives.")
        dessiner_pendu(6 - tentatives_restantes)
    else:
        message.set("Bonne lettre !")

    mot_affiche.set(afficher_mot_cache(mot_a_trouver, lettres_trouvees))

    if "_" not in mot_affiche.get():
        message.set("🎉 Félicitations, vous avez gagné !")
        bouton_verifier.config(state=tk.DISABLED)
        global parties_gagnees
        parties_gagnees += 1
    elif tentatives_restantes == 0:
        message.set(f"💀 Perdu ! Le mot était '{mot_a_trouver}'.")
        bouton_verifier.config(state=tk.DISABLED)
        global parties_perdues
        parties_perdues += 1


def nouvelle_partie():
    global mot_a_trouver, lettres_trouvees, mot_affiche, tentatives_restantes
    mot_a_trouver = choisir_mot()
    lettres_trouvees = set()
    tentatives_restantes = 6
    mot_affiche.set(afficher_mot_cache(mot_a_trouver, lettres_trouvees))
    message.set("Entrez une lettre pour deviner le mot.")
    bouton_verifier.config(state=tk.NORMAL)
    dessiner_support()

def afficher_scores():
    messagebox.showinfo("Scores", f"🏆 Parties gagnées : {parties_gagnees}\n💀 Parties perdues : {parties_perdues}")

def afficher_aide():
    regles = (
        "🎯 Règles du jeu du Pendu :\n\n"
        "- Un mot est choisi aléatoirement.\n"
        "- À chaque tour, vous proposez une lettre.\n"
        "- Si la lettre est dans le mot, elle est dévoilée.\n"
        "- Sinon, une partie du pendu est dessinée.\n"
        "- Vous avez 6 tentatives avant d'être pendu !\n"
        "- Le jeu se termine quand le mot est trouvé ou perdu."
    )
    messagebox.showinfo("Aide - Règles du jeu", regles)

def quitter_partie():
    global parties_perdues
    message.set(f"🚪 Partie abandonnée. Le mot était : '{mot_a_trouver}'.")
    mot_affiche.set(" ".join(mot_a_trouver))
    bouton_verifier.config(state=tk.DISABLED)
    parties_perdues += 1

#Interface 
root = tk.Tk()
root.title("Jeu du Pendu")
root.geometry("400x400")

mot_affiche = tk.StringVar()
message = tk.StringVar()

label_mot = tk.Label(root, textvariable=mot_affiche, font=("Arial", 16), anchor="center", justify="center")
label_mot.pack(pady=10)

message_du_jeu = tk.Label(root, textvariable=message, font=("Arial", 12), fg="red", anchor="center", justify="center")
message_du_jeu.pack(pady=5)

frame_input = tk.Frame(root)
frame_input.pack(pady=5)

entree_lettre = tk.Entry(frame_input, font=("Arial", 14), width=5, justify="center")
entree_lettre.pack(side=tk.LEFT, padx=5)

bouton_verifier = tk.Button(frame_input, text="Vérifier", command=verifier_lettre)
bouton_verifier.pack(side=tk.LEFT)

bouton_nouvelle_partie = tk.Button(root, text="Nouvelle Partie", command=nouvelle_partie)
bouton_nouvelle_partie.pack(pady=10)

bouton_aide = tk.Button(root, text="Aide", command=afficher_aide)
bouton_aide.pack(pady=5)

frame_boutons = tk.Frame(root)
frame_boutons.pack(pady=5, fill=tk.X)

bouton_scores = tk.Button(frame_boutons, text="Scores", command=afficher_scores)
bouton_scores.pack(side=tk.LEFT, expand=True, padx=5)

bouton_quitter_partie = tk.Button(frame_boutons, text="Quitter la partie", command=quitter_partie)
bouton_quitter_partie.pack(side=tk.RIGHT, expand=True, padx=5)


canvas = tk.Canvas(root, width=400, height=200, bg="white")
canvas.pack()

nouvelle_partie()

def touche_entree(event):
    verifier_lettre()

root.bind("<Return>", touche_entree)

root.mainloop()
